In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.preprocessing import MinMaxScaler

### Archivo CSV

In [2]:
dir_procesado = 'data/procesado/'
archivo = 'datos_crudos.csv'
df = pd.read_csv(f'{dir_procesado}{archivo}')

df.head(2)

,id,vendor,lat,lon,days_on_site,bathrooms,parking_lots,num_bedrooms,m2,final_price,price_square_meter
0,1,Investemx,19.421317,-99.164182,5.0,2.0,2,2.0,127.0,9000000.0,70866.141732
1,2,Avitana,19.421029,-99.163541,7.0,1.0,1,1.0,54.0,4264000.0,78962.962963


### Columna "vendor"

In [3]:
df_col = df.groupby('vendor').agg({'id':'count'})
df_col.reset_index(drop=False, inplace=True)
df_col.sort_values(['id'], ascending=False, inplace=True)

total_col = df_col['id'].sum()
df_col['total_porcentaje'] = df_col['id'].apply(lambda x: x/total_col*100)
df_col['total_porcentaje_acumulado'] = df_col['total_porcentaje'].cumsum()

In [4]:
df_col.head(10)

,vendor,id,total_porcentaje,total_porcentaje_acumulado
94,Metros Cuadrados Expertos,115,11.758691,11.758691
124,Rent A House Cdmx,111,11.349693,23.108384
13,Buscatuhogarmexico,95,9.713701,32.822086
11,Bienesraiceslomelin,42,4.294479,37.116564
74,Houm,39,3.987730,41.104294
140,Zona Ryg,36,3.680982,44.785276
2,Airnetwork Asesores Inmob,31,3.169734,47.955010
1,Aire Y Acero Inmobiliaria,31,3.169734,51.124744
76,Incitybienesraices,27,2.760736,53.885481
41,Consultores Patrimoniales,26,2.658487,56.543967


In [5]:
lista_col = df_col['vendor'].values[:5]
lista_col

array(['Metros Cuadrados Expertos', 'Rent A House Cdmx',
       'Buscatuhogarmexico', 'Bienesraiceslomelin', 'Houm'], dtype=object)

### Conjunto de datos para modelo

In [6]:
df_modelo = df.copy()
df_modelo['vendor_mod'] = df_modelo['vendor'].apply(lambda x: x if x in lista_col else 'otros')

In [7]:
df_modelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  978 non-null    int64  
 1   vendor              978 non-null    object 
 2   lat                 978 non-null    float64
 3   lon                 978 non-null    float64
 4   days_on_site        978 non-null    float64
 5   bathrooms           978 non-null    float64
 6   parking_lots        978 non-null    int64  
 7   num_bedrooms        978 non-null    float64
 8   m2                  978 non-null    float64
 9   final_price         978 non-null    float64
 10  price_square_meter  978 non-null    float64
 11  vendor_mod          978 non-null    object 
dtypes: float64(8), int64(2), object(2)
memory usage: 91.8+ KB


### One Hot Encoder

In [8]:
df_ohe = pd.get_dummies(df_modelo, columns=['vendor_mod'])
df_ohe.head(3)

,id,vendor,lat,lon,days_on_site,bathrooms,parking_lots,num_bedrooms,m2,final_price,price_square_meter,vendor_mod_Bienesraiceslomelin,vendor_mod_Buscatuhogarmexico,vendor_mod_Houm,vendor_mod_Metros Cuadrados Expertos,vendor_mod_Rent A House Cdmx,vendor_mod_otros
0,1,Investemx,19.421317,-99.164182,5.0,2.0,2,2.0,127.0,9000000.0,70866.141732,0,0,0,0,0,1
1,2,Avitana,19.421029,-99.163541,7.0,1.0,1,1.0,54.0,4264000.0,78962.962963,0,0,0,0,0,1
2,3,Ebranuncios,19.407814,-99.161872,4.0,3.0,2,3.0,163.0,8093500.0,49653.374233,0,0,0,0,0,1


### Conjunto de datos para modelo X^2

In [9]:
lista_x2 = [
    'days_on_site',
    'bathrooms',
    'parking_lots',
    'num_bedrooms',
    'm2']

In [10]:
df_sq = df_ohe.copy()
for col in lista_x2:
    df_sq[f'{col}_x2'] = df_sq[col].apply(lambda x: x**2)

In [11]:
df_sq.head(3)

,id,vendor,lat,lon,days_on_site,bathrooms,parking_lots,num_bedrooms,m2,final_price,...,vendor_mod_Buscatuhogarmexico,vendor_mod_Houm,vendor_mod_Metros Cuadrados Expertos,vendor_mod_Rent A House Cdmx,vendor_mod_otros,days_on_site_x2,bathrooms_x2,parking_lots_x2,num_bedrooms_x2,m2_x2
0,1,Investemx,19.421317,-99.164182,5.0,2.0,2,2.0,127.0,9000000.0,...,0,0,0,0,1,25.0,4.0,4,4.0,16129.0
1,2,Avitana,19.421029,-99.163541,7.0,1.0,1,1.0,54.0,4264000.0,...,0,0,0,0,1,49.0,1.0,1,1.0,2916.0
2,3,Ebranuncios,19.407814,-99.161872,4.0,3.0,2,3.0,163.0,8093500.0,...,0,0,0,0,1,16.0,9.0,4,9.0,26569.0


### Conjunto de datos escalado

In [12]:
df_sc = df_sq.copy()

lista_escala = [
    'days_on_site',
    'bathrooms',
    'parking_lots',
    'num_bedrooms',
    'm2',
    'days_on_site_x2',
    'bathrooms_x2',
    'parking_lots_x2',
    'num_bedrooms_x2',
    'm2_x2']

for col in lista_escala:
    scaler = MinMaxScaler()
    df_sc[f'{col}_sc'] = scaler.fit_transform(df_sc[[col]])

In [13]:
df_sc.head(3)

,id,vendor,lat,lon,days_on_site,bathrooms,parking_lots,num_bedrooms,m2,final_price,...,days_on_site_sc,bathrooms_sc,parking_lots_sc,num_bedrooms_sc,m2_sc,days_on_site_x2_sc,bathrooms_x2_sc,parking_lots_x2_sc,num_bedrooms_x2_sc,m2_x2_sc
0,1,Investemx,19.421317,-99.164182,5.0,2.0,2,2.0,127.0,9000000.0,...,0.005487,0.045455,0.5,0.333333,0.366412,0.000045,0.005682,0.375,0.200000,0.178682
1,2,Avitana,19.421029,-99.163541,7.0,1.0,1,1.0,54.0,4264000.0,...,0.008230,0.000000,0.0,0.000000,0.087786,0.000090,0.000000,0.000,0.000000,0.023030
2,3,Ebranuncios,19.407814,-99.161872,4.0,3.0,2,3.0,163.0,8093500.0,...,0.004115,0.090909,0.5,0.666667,0.503817,0.000028,0.015152,0.375,0.533333,0.301668


### Datos entrenamiento y evaluación

In [14]:
X = df_sc[[
    'vendor_mod_Bienesraiceslomelin',
    'vendor_mod_Buscatuhogarmexico',
    'vendor_mod_Houm',
    'vendor_mod_Metros Cuadrados Expertos',
    'vendor_mod_Rent A House Cdmx',
    'vendor_mod_otros',
    'days_on_site_sc',
    'bathrooms_sc',
    'parking_lots_sc',
    'num_bedrooms_sc',
    'm2_sc',
    'days_on_site_x2_sc',
    'bathrooms_x2_sc',
    'parking_lots_x2_sc',
    'num_bedrooms_x2_sc',
    'm2_x2_sc']]

y = df_sc['price_square_meter']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=75)

### Modelo base (regresión lineal)

In [16]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

### Entrenamiento

In [17]:
y_pred = model.predict(X_train)

In [18]:
r2_score(y_train, y_pred)

0.30059007840683327

In [19]:
print("{:,.2f}".format(mean_squared_error(y_train, y_pred)))

253,987,040.17


### Evaluación

In [20]:
y_pred = model.predict(X_test)

In [21]:
r2_score(y_test, y_pred)

0.2585038185149111

In [22]:
print("{:,.2f}".format(mean_squared_error(y_test, y_pred)))

261,235,353.67
